# Address-AutoComplete-Bot

This notebook shows the end-to-end process of training and building a Address Autocomplete Bot using Global Attention Mechanism.

## Table of Contents

- [1 - Idea](#1)
- [2 - Review](#2)
- [3 - Address Datasets](#3)
    - [3.1 - Training Data Generation](#3-1)
    - [3.2 - Data Processing](#3-2)
- [4 - Encoder, Attention, and Decoder](#4)
    - [4.1 - Encoder](#4-1)
    - [4.2 - Attention](#4-2)

<a id="1"></a>
## 1 - Idea

Address is one of the most important pieces of data to get right for all transactions in life. When we input the correct address into a website, we ensure that the products are delivered to the right address, improve customer satisfaction, and reduce the risk of fraud. My wife and I buy hundreds of items across dozens of websites, and it's hard to imagine what would happen if the address was incorrect. Luckily, Google has services like the [Maps API](https://developers.google.com/maps/documentation/javascript), which eliminates these concerns. These map APIs are widely available, lightweight, and easy to deploy. They can be used to build dynamic and interactive maps for web applications using geospatial data, ensuring that customers never enter the wrong address again. For example, the [Place Autocomplete Address Form](https://developers.google.com/maps/documentation/javascript/examples/places-autocomplete-addressform) helps to accurately supply address details. The Place Autocomplete Address Form sample captures selected address components from the Google Places database, and uses them to populate an address form.

Recently, I completed the the [Deep Learning Specialization](https://www.deeplearning.ai/courses/deep-learning-specialization/) class from Andrew NG. In this class, I learned about the attention mechanism. The attention mechanism is a powerful neural network technique that has revolutionized the field of Natural Language Processing (NLP). It allows models to focus on specific parts of their input data, which is essential for learning long-range dependencies and understanding the context of complex sequences. Attention mechanisms have been shown to significantly improve the performance of Large Language Models (LLMs) on a variety of tasks, including machine translation, text summarization, and question answering.

One question that arises is whether it is possible to train an LLM to predict accurate and complete addresses without requiring any geospatial knowledge. While it is known that LLMs are capable of learning complex and non-linear relationships between features and predictions on tabular data, it is unclear whether they can perform well on address correction (i.e., text generation) without knowing the meaning of individual address components such as street names, cities, and states.

This project aims to develop an address autocomplete bot that uses the attention mechanism to autocomplete addresses without requiring any geospatial knowledge. The bot will be trained on a large dataset of correctly formatted addresses, and will use the attention mechanism to learn the relationships between different address components. When given an inaccurate address, the bot will be able to detect the errors and autocomplete the address by filling in missing components or correcting incorrect components.

*References: [Effective Approaches to Attention-based Neural Machine Translation (Luong et al., 2015)](https://arxiv.org/abs/1508.04025v5).*

<a id="2"></a>
## 2 - Review

Deep learning methods are being used in solving NLP quesitons since the early 2000s. Deep learning methods are a type of neural network that can learn complex relationships in data. In 2010s, we witness the rise of large language models (LLMs). LLMs are trained on massive datasets of text and code, which allows them to learn the statistical relationships between words and phrases at an unprecedented scale. In the paragraph below, I will summarize the development of deep learning methods on NLP, from Recurrent Neural Network (RNN), Long Short-Term Memory (LSTM), to the recent developed Attention Mechanism. 

- **Recurrent Neural Networks (RNNs)**

RNN are a type of neural network that can process sequences of data. They are the first type of neural networks that are able to learn long-term dependencies within text data, making them the first choice to solve NLP problems. However, one of the disadvantages of RNN is that they are prone to "vanishing gradient". The vanishing gradient problem is a major obstacle to training RNNs to learn long-term dependencies. This is because long-term dependencies require the network to remember information from many time steps ago, and the vanishing gradient problem makes it difficult for the network to do this.

A language example can easily demonstrate this idea of long-term dependencies. In the sentence below, the verb `has` is influenced by the word `Dog` at the very beginning. If the word `Dog` is changed to a plural form, then the verb `has` would need to be updated to the word `have`. This long-range dependencies can extend over a very long step, as we do not know how many words (aka steps) are in between these two words.

<div style="text-align: center;">
<img src="image/language_example.png" width="500"> <br>
<caption><center><b>Figure 1</b>: A language example </center></caption>
</div>

A standard RNN neural network architecture is shown below. In order to learn the long-term dependencies relationship, we hope the model is able to pass on the hidden state $a^{\langle 2 \rangle}$ to the position of $x^{\langle T_{x} \rangle}$. The basic RNNs that we have seen so far are not very good at handling such long-term dependencies, mainly due to the Vanishing Gradient Problem. When the gradients traverse through multiple steps it become very small, which makes it difficult for the network to learn.

<div style="text-align: center;">
<img src="image/rnn.png" width="500"> <br>
<caption><center><b>Figure 2</b>: Basic RNN Structure </center></caption>
</div>

- **Long Short-Term Memory (LSTM)**

LSTMs are a type of RNN that address the vanishing gradient problem. They do this by using gates to control the flow of information in the network. The figure below shows the operations of an LSTM cell.

The gates in a LSTM cell are described below:

- Forget Gate ($\mathbf{\Gamma}_{f}$): The forget gate can be used to *"forget"* the previous state. For example, if the subject changes from a singular word `Dog` to a plural `Dogs`, the memory of the previous state becomes outdated and should be forgotten. 

- Update Gate ($\mathbf{\Gamma}_{i}$): The update gate can be used to decide what aspects of the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ to add to the cell state $c^{\langle t \rangle}$. The candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ is a tensor containing information from the current time step that **may** be stored in the current cell state $\mathbf{c}^{\langle t \rangle}$. The current cell state $\mathbf{c}^{\langle t \rangle}$ is the "memory" that gets passed onto future time steps.

- Output Gate ($\mathbf{\Gamma}_{o}$): The output gate decides what gets sent as the prediction (output) of the time step.

The LSTM unit can remember long-term dependencies because the cell state is not updated at every time step, as shown in formula 4. This allows the LSTM unit to retain information from previous time steps, even if it is not used in the current time step.

<div style="text-align: center;">
<img src="image/lstm.png" width="500"> <br>
<caption><center><b>Figure 3</b>: LSTM Cell </center></caption>
</div>

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$
$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 
$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$
$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$
$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$ 
$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

- **Attention Mechanism**

LSTM solves the vanishing gradient problem as it allows the cell state to flow through multiple time steps. However, in the language example above, the word `has` does not rely on any other word in the sentence other than the word `dog`. Attention Mechanism was developed to allow the neural network to put more weight on certain long-range dependencies than others at each position $x^{\langle T_{x} \rangle}$, hence the word "Attention".

The attention mechanism was first proposed by __[Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et al., 2014)](https://arxiv.org/abs/1409.0473)__, and its a powerful tool that has revolutionized the field of NLP.  Attention Mechanism is commonly used with the encoder-decoders network family. The encoder encodes a source sentence into a fixed-length tensor. The attention mechanism then calcualtes a score for each word, indicating how important the word is to the current position $x^{\langle T_{x} \rangle}$. When the score is higher, the network will pay more attention to this word for the current output $y^{\langle T_{x} \rangle}$. When the score is lower, the network pay less attention.

The figure below shows what one "attention" step does to calculate the attention variables $\alpha^{\langle t, t' \rangle}$. $s^{\langle t-1 \rangle}$ is the one-step prior hidden state from the post-attention decoder, and $a^{\langle t' \rangle}$ is the hidden state from the pre-attention encoder. $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ are fed into a simple neural network with a dense layer to learn and compute the output $e^{\langle t, t' \rangle}$. $e^{\langle t, t' \rangle}$ is then used when computing the attention $\alpha^{\langle t, t' \rangle}$ that $y^{\langle t \rangle}$ should pay to $a^{\langle t' \rangle}$. 

Finally, the $context^{ \langle t \rangle }$ works as a weighted average of all the attention weights. Then, the decoder's output along with the context vector is used to predict the next output $y^{\langle T_{x+1} \rangle}$

$$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 

<div style="text-align: center;">
<img src="image/attn_mechanism.png" width="500"> <br>
<caption><center><b>Figure 4</b>: Attention Mechanism </center></caption>
</div>

<a id="3"></a>
## 3 - Datasets

The address data is downloaded from the [New York State Geographic Information System (GIS)](https://gis.ny.gov/) website. We will only train this bot on the larger NY area addresses (Manhattan, Brooklyn, Queens, Bronx, Staten Island) for demonstration purpose and to minimize network training time.
The address dataset download direction can be found [here](https://gis.ny.gov/system/files/documents/2023/03/how-to-create-county-filters-of-nys-address-point-data.pdf).

The dataset has the below columns: 

|**Column Name**|**Type**|
|------|------|
|AddressNumber|int|
|StreetName|string|
|PostType|string|
|CountyName|string|
|CityTownName|string|
|ZipName|string|
|ZipCode|int|
|State|string|

In [2]:
import pandas as pd
import numpy as np
import utils
import tensorflow as tf
import tensorflow_text as tf_text

c:\Users\gkchen\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
ny_address = pd.read_parquet('Datasets/NYS_clean.parquet.gz', engine='pyarrow')
print(f'Dataset shape: {ny_address.shape}')
ny_address.sample(5)

Dataset shape: (746396, 8)


,AddressNumber,StreetName,PostType,CountyName,CityTownName,ZipName,ZipCode,State
5310541,64,50,Ave,Queens,New York,Corona,11368,NY
4351709,9,125,St,Queens,New York,South Richmond Hill,11419,NY
5410919,321,34,St,New York,New York,New York,10016,NY
5399431,430,84,St,New York,New York,New York,10028,NY
5507966,742,43,St,Kings,New York,Brooklyn,11203,NY


In [9]:
# Checking the County Distribution: 62% of the addresses are in Kings (Brooklyn borough) and Quees (Queens borough) county.
ny_address['CountyName'].value_counts(normalize=True)

Kings          0.386011
Queens         0.234193
Richmond       0.166930
Bronx          0.136503
New York       0.076250
Nassau         0.000074
Westchester    0.000039
Name: CountyName, dtype: float64

In [11]:
# Checking the length for each address component. Max word length is an important parameter for the word embedding that will happen later
for col_i in ny_address.columns:
    print(f'{col_i} has max length of {ny_address[col_i].astype(str).apply(len).max()}')

AddressNumber has max length of 5
StreetName has max length of 28
PostType has max length of 6
CountyName has max length of 11
CityTownName has max length of 8
ZipName has max length of 19
ZipCode has max length of 5
State has max length of 2


<a id="3-1"></a>
## 3-1 - Training Data Generation

In [12]:
import utils
from importlib import reload
utils = reload(utils)

c:\Users\gkchen\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


To train the neural network to output the correct complete address, we need to teach it the relationships between different address components. To do this, we generate training data by "masking" certain address components. The `context_raw` dataframe contains the "masked" addresses, and the `target_raw` dataframe contains the complete addresses. We sample with replacement so that one address can be used for multiple training data points.

The figure below illustrates the training data generation process. For example, if we mask one complete address three different ways, we will generate three training data points.

One limitation of this method is that not all training data points are equally helpful to the model. This is because the address components have a hierarchical structure. For example, training data point No. 3 will be helpful to the model because we want it to learn to output the most likely zip code for `321 34th St, New York, New York`, which is `10016`. However, training data point No. 1 will not be as helpful, because many addresses can have the zip code `10016`. We expect the model cost function to reduce with training data point No. 3, but not so much with training data point No. 1. 

<div style="text-align: center;">
<img src="image/mask_generate.PNG" width="800"> <br>
<caption><center><b>Figure 5</b>: Training Data Generation Process </center></caption>
</div>

In [14]:
context_raw,target_raw = utils.create_label_target('Datasets/NYS_clean.parquet.gz')
print(f'Context shape: {context_raw.shape}')
print(f'Target shape: {target_raw.shape}')

Context shape: (2000000,)
Target shape: (2000000,)


In [18]:
random_numbers = np.random.choice(len(context_raw), 1)[0]
print(f'Masked Address: {context_raw[random_numbers]}')
print(f'Complete Address: {target_raw[random_numbers]}')


Masked Address: 4015 Church Kings NY 11203
Complete Address: 4015 Church Ave Brooklyn Kings NY 11203


In [3]:
# np.save("Datasets/context_raw.npy", context_raw)
# np.save("Datasets/target_raw.npy", target_raw)

context_raw = np.load("Datasets/context_raw.npy")
target_raw = np.load("Datasets/target_raw.npy")

<a id="3-2"></a>
## 3-2 - Data Processing

**Step 1**: Split the dataset into 80% `train_raw` and 20% `val_raw`. Create a `tf.data.Dataset` for these strings. `tf.data.Dataset` is designed to be efficient, both in terms of memory usage and runtime performance. It can automatically parallelize data loading and processing, and it can also be used to cache data in memory or on disk to improve performance.

In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

**Step 2**: We can use the `tf.keras.layers.TextVectorization` to apply preprocessing to the text, and maps text features to integer token sequences. The text standardization is handled by the `utils.tf_lower_and_split_punct`.

In [5]:
# This parameters defines the saximum size of the vocabulary for this layer
max_vocab_size = 5000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=utils.tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

# The adapt method reads one epoch of the training data, and works a lot like Model.fit. This method initializes the layer based on the data.
context_text_processor.adapt(train_raw.map(lambda context, target: context))

In [6]:
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=utils.tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target: target))

In [31]:
# The context_text_processor convert a address into a token sequences
# The [START] and [END] represents the Start of String token and End of String token. These tokens will tell the model when to start and stop the prediction.
example_text = tf.constant('68 96 St Brooklyn Kings NY 11212')
example_tokens = context_text_processor(example_text)
print(f'Address To token: {example_tokens}')

context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens.numpy()]
print('Token back to Address')
print(' '.join(tokens))


Address To token: [  2 141 218   5   7   6   4 128   3]
Token back to Address
[START] 68 96 st brooklyn kings ny 11212 [END]


**Step 3**: Convert the `tf.data.Dataset` to 0-padded tensors of token. The `target_in` and `target_out` is shifted by one step relative to each other. 

In [8]:
def process_text(context, target):
    context = context_text_processor(context).to_tensor()
    target = target_text_processor(target)
    targ_in = target[:,:-1].to_tensor()
    targ_out = target[:,1:].to_tensor()
    return (context, targ_in), targ_out

train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

In [9]:
tf.data.Dataset.save(train_ds, "Datasets/train_ds.tfds")
tf.data.Dataset.save(val_ds, "Datasets/val_ds.tfds")

<a id="4"></a>
# 4 - Encoder, Attention, and Decoder

<a id="4-1"></a>
## 4-1 - Encoder

The goal of the encoder is to process the context sequence $x^{\langle T_{x} \rangle}$ (in our case, the address string) into a sequence of vectors that the decoder can use to predict the next output $y^{\langle T_{x+1} \rangle}$.
<br>
<br>
The details of the encoder network is listed below:

1. `tf.keras.layers.Embedding`: An embedding layer that convert postive integers (i.e., tokens) into dense vectors

2. `tf.keras.layers.Bidirectional`: A bi-directional wrapper for RNNs. Here we use a LSTM unit with 256 neurons. The reason we choose a bi-directional LSTM is because the hidden state should flow both ways in the encoder. For example, in the exmample address of `4015 Church Ave Brooklyn Kings NY 11203`, the borough component `Brooklyn` should have the hidden state flow from the prior component `4015 Church Ave` and also from the post component `Kings NY 11203`.


In [10]:
UNITS = 256

class Encoder(tf.keras.layers.Layer):
    def __init__(self, text_processor, units):
        super(Encoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(self.vocab_size, units, mask_zero=True)

        self.rnn = tf.keras.layers.Bidirectional(
            merge_mode='sum',
            layer=tf.keras.layers.LSTM(units,return_sequences=True))

    def call(self, x):
        shape_checker = utils.ShapeChecker()
        shape_checker(x, 'batch s')

        x = self.embedding(x)
        shape_checker(x, 'batch s units')

        x = self.rnn(x)
        shape_checker(x, 'batch s units')

        return x

    def convert_input(self, texts):
        texts = tf.convert_to_tensor(texts)
        if len(texts.shape) == 0:
          texts = tf.convert_to_tensor(texts)[tf.newaxis]
        context = self.text_processor(texts).to_tensor()
        context = self(context)
        return context

<a id="4-2"></a>
## 4-2 - Attention

The attention layer allows the decoder to access the information extracted by the encoder. It computes a `context vector` from the entire context sequence, and adds that to the decoder's output.
<br>
<br>
The details of the attention layer is listed below:

1. `tf.keras.layers.MultiHeadAttention`: A single head attention layer (num_heads = 1). The `key_dim` is set to 256. This means the attention head for the `query` and `key` will have dimension = 256.

* The components of the attention layer is specified here:
    * (`Batch`, $m$): Number of records in each batch  
    * (`t`, $t$): Target dimension
    * (`s`, $s$): Source dimension
    * (`heads`, $h$): Number of attention heads
    * (`units`, $dim$): Number of hidden units from the LSTM layer

* The matrices dimension for each attention component:
    * `Query`:  ($m$,$t$,$dim$)
    * `Value`:  ($m$,$s$,$dim$)
    * `Attention Output`: ?
    * `Attention Score (before average across heads)`: ($m$, $h$, $t$, $s$)
    * `Attention Score (after average across heads)`: ($m$, $t$, $s$)

2. `tf.keras.layers.Add`: Concatting the attention output along with the input `X`

3. `tf.keras.layers.LayerNormalization`: Normalize the activations of the previous layer for each given example in a batch independently, not across examples.

In [11]:
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x, context):
        shape_checker = utils.ShapeChecker()

        shape_checker(x, 'batch t units')
        shape_checker(context, 'batch s units')

        attn_output, attn_scores = self.mha(
            query=x,
            value=context,
            return_attention_scores=True)

        shape_checker(x, 'batch t units')
        shape_checker(attn_scores, 'batch heads t s')

        attn_scores = tf.reduce_mean(attn_scores, axis=1)
        shape_checker(attn_scores, 'batch t s')
        self.last_attention_weights = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x

<a id="4-3"></a>
## 4-3 - Decoder

The goal of the decoder is to generate predictions for the next token at each location in the target sequence.
<br>
<br>
The details of the decoder is listed below:

1. 

In [ ]:
class Decoder(tf.keras.layers.Layer):
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, text_processor, units):
        super(Decoder, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.word_to_id = tf.keras.layers.StringLookup(
            vocabulary=text_processor.get_vocabulary(),
            mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(
            vocabulary=text_processor.get_vocabulary(),
            mask_token='', oov_token='[UNK]',
            invert=True)
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')

        self.units = units


        # 1. The embedding layer converts token IDs to vectors
        self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                                units, mask_zero=True)

        # 2. The RNN keeps track of what's been generated so far.
        self.rnn = tf.keras.layers.GRU(units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')

        # 3. The RNN output will be the query for the attention layer.
        self.attention = CrossAttention(units)

        # 4. This fully connected layer produces the logits for each
        # output token.
        self.output_layer = tf.keras.layers.Dense(self.vocab_size)

@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
    shape_checker = utils.ShapeChecker()
    shape_checker(x, 'batch t')
    shape_checker(context, 'batch s units')

    # 1. Lookup the embeddings
    x = self.embedding(x)
    shape_checker(x, 'batch t units')

    # 2. Process the target sequence.
    x, state = self.rnn(x, initial_state=state)
    shape_checker(x, 'batch t units')

    # 3. Use the RNN output as the query for the attention over the context.
    x = self.attention(x, context)
    self.last_attention_weights = self.attention.last_attention_weights
    shape_checker(x, 'batch t units')
    shape_checker(self.last_attention_weights, 'batch t s')

    # Step 4. Generate logit predictions for the next token.
    logits = self.output_layer(x)
    shape_checker(logits, 'batch t target_vocab_size')

    if return_state:
        return logits, state
    else:
        return logits